### General approach

Our goal is to construct a low-rank approximation of a data matrix $A$ of DNA methylation patterns (with many missing data points). Each row of $A$ contains a binary sequence of methylation, $A_{ij} = 1$ for methylated sites, $A_{ij} = 0$ for unmethylated sites, and $A_{ij} = \text{NA}$ when site $j$ is unobserved in read $i$. We factor $A$ as a product of two matrices $X$ and $Y$:

$$
A = XY \\ A \in \mathbf{R}^{m \times n},~~ X \in \mathbf{R}^{m \times k},~~ Y \in \mathbf{R}^{k \times n}
$$

That is, we have a dataset $m$ reads that provide partial information over $n$ methylation-eligible sites of interest (e.g. over a selected number of genes, or, more ambitiously, over the entire genome). We suppose that there are no more than $k$ types of cells with distinct methylation patterns.

The rows of $Y$ provide the estimates for the $k$ distinct methylation patterns. A row of $X$ assigns the relative probabilities for that read to one of the $k$ patterns. An interesting question is whether we believe the rows of $X$ to be sparse. A sparse rowspace of $X$ would correspond to there being distinct methylation patterns across cell types, with little to no mixed patterns. A non-sparse rowspace would allow for a spectrum of different DNA methylation profiles.

### Optimization Problems

We use $x_i$ to denote row $i$ of $X$, and $y_j$ to denote column $j$ of $Y$, so that $x_i y_j$ is a vector dot product. Let $(i,j) \in \Omega$ denote the indices of observed entries in $A$. Then a low-rank approximation can be achieved by solving the unconstrained optimization problem:

$$
\begin{aligned}
& \underset{X,Y}{\text{minimize}}
& & \sum_{(i,j) \in \Omega} \log(1+\exp(-A_{ij} (x_i y_j)) - \gamma ||X||_F^2 - \gamma||Y||_F^2 
\end{aligned}
$$

The first term is a logistic loss function. The last two are regularization terms that penalize the *Frobenius norm* of the matrices. [Udell et al. (2014)](http://web.stanford.edu/~udell/doc/glrm.pdf) show that this problem is equivalent to the rank-constrained nuclear-norm regularized problem:

$$
\begin{aligned}
& \underset{X,Y}{\text{minimize}}
& & \sum_{(i,j) \in \Omega} \log(1+\exp(-A_{ij} (x_i y_j)) - 2 \gamma ||XY||_*^2 \\
& \text{subject to}
& & \text{rank}(Z) \leq k
\end{aligned}
$$

as long as $k$ is chosen to be larger than the true rank of $A$. The *Frobenius norm* and *nuclear norm* are respectively defined as:

$$
||X||_F = \sqrt{\sum_{i=1}^k \sigma_i^2} = \sqrt{ \text{trace} \left ( X^T X \right ) } \\
||X||_* = \sum_{i=1}^k \sigma_i = \text{trace} \left ( \sqrt{X^T X} \right )
$$

### Relevant Papers on Matrix Completion

Emmanuel J. Candès, Benjamin Recht. [Exact Matrix Completion via Convex Optimization](http://dx.doi.org/10.1007/s10208-009-9045-5). *Found Comput Math* 9, 717–772 (2009)

Emmanuel J. Candès, Terence Tao. [The Power of Convex Relaxation: Near-Optimal Matrix Completion](http://dx.doi.org/10.1109/tit.2010.2044061). *IEEE Transactions on Information Theory* 56, 2053–2080 (2010).

Emmanuel J Candès, Yaniv Plan. [Matrix Completion With Noise](http://dx.doi.org/10.1109/jproc.2009.2035722). *Proc. IEEE* 98, 925–936 (2010).

Benjamin Recht, Maryam Fazel, Pablo A. Parrilo. [Guaranteed Minimum-Rank Solutions of Linear Matrix Equations via Nuclear Norm Minimization](http://dx.doi.org/10.1137/070697835). *SIAM Rev.* 52, 471–501 (2010).

In [1]:
using LowRankModels
# problem dimensions
m,n,k = 100,100,5
# generate clustered data
Y = randn(k,n)
A = zeros(m,n)
for i=1:m
    A[i,:] = Y[mod(i,k)+1,:]
end
# quadratic loss
losses = fill(quadratic(),n)
# set regularization: x is 1-sparse, y is not regularized
rx = onesparse()
ry = zeroreg()
# form GLRM
glrm = GLRM(A,losses,rx,ry,k)

GLRM(100x100 Array{Float64,2}:
 -0.0587522   0.804001  0.496302  …  -0.290197   -0.413114  -0.877346
 -0.798089    2.23288   1.83455       0.966752   -0.677168  -0.656077
  1.39135    -0.737739  0.9028       -0.116393   -0.941991   1.70175 
  0.550548    0.522705  0.907778     -0.485389   -1.35808   -2.5533  
  0.442172    0.239211  0.805606     -0.0264728  -0.913191   1.00261 
 -0.0587522   0.804001  0.496302  …  -0.290197   -0.413114  -0.877346
 -0.798089    2.23288   1.83455       0.966752   -0.677168  -0.656077
  1.39135    -0.737739  0.9028       -0.116393   -0.941991   1.70175 
  0.550548    0.522705  0.907778     -0.485389   -1.35808   -2.5533  
  0.442172    0.239211  0.805606     -0.0264728  -0.913191   1.00261 
 -0.0587522   0.804001  0.496302  …  -0.290197   -0.413114  -0.877346
 -0.798089    2.23288   1.83455       0.966752   -0.677168  -0.656077
  1.39135    -0.737739  0.9028       -0.116393   -0.941991   1.70175 
  ⋮                               ⋱                        